In [80]:
%matplotlib inline

import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from ipywidgets import interact
import matplotlib.patches as patches
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import *
import pandas as pd

import json

In [81]:
root_path = 'sample' # images folder 
anno_root = 'info.json' # json file folder 

In [82]:
with open(anno_root, 'r') as f:
    train_json = json.load(f)
    images = train_json['images']
    annotations = train_json['annotations']

images_viz = dict()
for item in images:
    images_viz[item['id']] = dict()
    images_viz[item['id']]['id'] = item['id']
    images_viz[item['id']]['file_name'] = item['file']

In [84]:
for anno in annotations:
    images_viz[anno['image_id']]['bbox'] = anno['ocr']['word']

In [85]:
categories = {
            "0": "UNKNOWN",
            "1": "name",
            "2": "phone",
            "3": "email",
            "4": "position",
            "5": "company",
            "6": "department",
            "7": "address",
            "8": "site",
            "9": "account",
            "10": "wise",
        }

In [86]:
palette = sns.color_palette('bright', 11)

fnames = [(images_viz[id]['id'], images_viz[id]['file_name']) for id in images_viz]

@interact(idx=(fnames[0][0], fnames[-1][0]), cls_id=range(0, len(categories)+1))
def showImg(idx=0, cls_id=len(categories)):
    fig, ax = plt.subplots(dpi=200)
    img = io.imread(root_path + '/' +fnames[idx][1])

    anns = images_viz[idx]['bbox']

    ax.imshow(img)
    for ann in anns:
        class_idx = ann['category_id']

        ax.set_title(f"{fnames[idx][1]}", fontsize=7)

        # 축 제거 
        ax.set_xticks([])
        ax.set_yticks([])
        for pos in ['right', 'top', 'bottom', 'left']:
            ax.spines[pos].set_visible(False)

        points = np.array(ann['points'])
        text = ann['text']

        # bbox 시각화 
        if(class_idx == cls_id or cls_id == 11):
            color = palette[class_idx]
            ax.add_patch(
                patches.Polygon(
                    points,
                    closed=True,
                    edgecolor=color,
                    fill=False,
                    ),
                )

            x, y = points[-1][0], points[0][1]

            text_y = y-5 if y>5 else y+5 
            plt_text = ax.text(x,text_y, f'{class_idx} : {categories[str(class_idx)]}', color='white', fontsize='3', weight='semibold', backgroundcolor=color)
            plt_text.set_bbox(dict(
                facecolor=palette[class_idx],  # background color
                alpha=0.6,  # background alpha
                edgecolor='none',  # border color
                pad=2
            ))

interactive(children=(IntSlider(value=0, description='idx', max=2), Dropdown(description='cls_id', index=11, o…